In [16]:
import datetime
import pandas as pd 
import numpy as np 
import os
import glob 
import matplotlib.pyplot as plt
import itertools
from attrs import define, field

## Strategy

In [17]:
def compute_returns(prices): 
    return (prices - prices.shift(1)) / prices

def compute_moving_average(returns, window): 
    return returns.rolling(window).mean()

def compute_signal(returns, signal_function, *args): 
    signal = signal_function(returns, *args)
    return signal

def compute_position(signal, position_scale = 0.1): 
    return signal * position_scale

def compute_strategy_returns(returns, positions): 
    return 1 + (returns * positions.shift(1)).sum(axis = 1)

def compute_strategy(prices, signal_function, *args): 
    returns = compute_returns(prices)
    signal = compute_signal(returns, signal_function, *args)
    positions = compute_position(signal)
    strategy_returns = compute_strategy_returns(returns, positions)

    all_days = strategy_returns.index

    level = 100
    all_levels = {all_days[0]: level}
    for day in all_days[1:]: 
        level *= strategy_returns[day]
        all_levels[day] = level
    return pd.Series(all_levels)

## Signal Options

In [18]:
def return_signal(returns): 
    signal = returns.map(lambda x: 1 if x > 0 else (0 if x == 0 else -1))
    return signal

def average_return_signal(returns, window): 
    average_returns = compute_moving_average(returns, window)
    signal = average_returns.map(lambda x: 1 if x > 0 else (0 if x == 0 else -1))
    return signal

def crossing_averages_signal(returns, short_window, long_window): 
    short_ma = compute_moving_average(returns, short_window)
    long_ma = compute_moving_average(returns, long_window)
    difference = short_ma - long_ma
    signal = difference.map(lambda x: 1 if x > 0 else (0 if x == 0 else -1))
    return signal

def selective_return_signal(returns, window, num_chosen): 
    average_returns = compute_moving_average(returns, window)
    ranked_returns = average_returns.rank(axis = 1, ascending = False)
    signal = ranked_returns.map(lambda x: 1 if x < num_chosen + 1 else -1)
    return signal

def selective_return_long_only_signal(returns, window, num_chosen): 
    average_returns = compute_moving_average(returns, window)
    ranked_returns = average_returns.rank(axis = 1, ascending = False)
    signal = ranked_returns.map(lambda x: 1 if x < num_chosen + 1 else 0)
    return signal

## Analytics

In [24]:
def sharpe_ratio(levels: pd.Series): 
    returns = np.log(levels).diff()
    volatility = returns.std() * np.sqrt(252)
    return returns.mean() * 252 / volatility

def max_drawdown(levels: pd.Series): 
    levels_list = levels.tolist()
    min_idx = np.argmax(np.maximum.accumulate(levels_list) - levels_list) 
    max_idx = np.argmax(levels_list[:min_idx])
    min_val = levels_list[min_idx]
    max_val = levels_list[max_idx]
    return (max_val - min_val) / max_val * 100
    

In [20]:
def test_parameters_for_strategy(prices, signal_function, *args): 
    records = {}
    max_sharpe_ratio = -10 # Initialise to low value
    max_key = None
    parameter_combinations = list(itertools.product(*args))
    for combination in parameter_combinations: 
        levels = compute_strategy(prices, signal_function, *combination)
        sharpe = sharpe_ratio(levels)
        mdd = max_drawdown(levels)
        records[combination] = [sharpe, mdd]
        if sharpe > max_sharpe_ratio: 
            max_sharpe_ratio = sharpe
            max_key = combination
    return records, max_sharpe_ratio, max_key

### Testing 

In [21]:
prices = pd.read_pickle(r"input_prices.pkl")
prices.head()

,AAPL,AMZN,GOOGL,MSFT
2004-08-19,0.4626,1.9269,2.5099,16.9629
2004-08-20,0.4639,1.9708,2.7094,17.0129
2004-08-23,0.4681,1.9678,2.7367,17.0879
2004-08-24,0.4812,1.9478,2.6233,17.0879
2004-08-25,0.4978,2.0102,2.6516,17.2824


In [28]:
@define(kw_only = True)
class StrategyTest: 
    uid: str
    signal_func: object
    parameters: list = []

    all_records: dict = field(init = False, default = None)
    max_sharpe_ratio: float = field(init = False, default = None)
    max_key: list = field(init = False, default = None)

    def test_parameters(self, prices): 
        self.all_records, self.max_sharpe_ratio, self.max_key = test_parameters_for_strategy(prices, self.signal_func, *self.parameters)

    def compute_strategy_with_max_parameters(self, prices): 
        return compute_strategy(prices, self.signal_func, *self.max_key)

strategies = [
                StrategyTest(uid = 'ReturnsOnly', 
                             signal_func = return_signal), 
                StrategyTest(uid = 'AverageReturn', 
                             signal_func = average_return_signal, 
                             parameters = [[5, 10, 25, 50, 100, 252]]), 
                StrategyTest(uid = 'CrossingAverages',
                             signal_func = crossing_averages_signal, 
                             parameters = [[5, 10, 25, 50], [100, 252, 500]]), 
                StrategyTest(uid = 'SelectiveReturns', 
                             signal_func = selective_return_signal, 
                             parameters = [[5, 10, 25, 50, 100, 252], [1, 2, 3]]), 
                StrategyTest(uid = 'SelectiveReturnsLongOnly', 
                             signal_func = selective_return_long_only_signal, 
                             parameters = [[5, 10, 25, 50, 100, 252], [1, 2, 3]]),
]

In [34]:
# Compute optimal parameters 
for strategy in strategies: 
    strategy.test_parameters(prices = prices)
    
    print(f'Strategy {strategy.uid} has maximum Sharpe Ratio of {strategy.max_sharpe_ratio} with key {strategy.max_key}')
    print(f'These parameters yield a max drawdown of {strategy.all_records[strategy.max_key][1]}')
    print('')

Strategy ReturnsOnly has maximum Sharpe Ratio of -0.24704327177049265 with key ()
These parameters yield a max drawdown of 44.5815291558671

Strategy AverageReturn has maximum Sharpe Ratio of 0.27370723392210256 with key (25,)
These parameters yield a max drawdown of 15.976142824143706

Strategy CrossingAverages has maximum Sharpe Ratio of 0.15882108232565645 with key (25, 100)
These parameters yield a max drawdown of 18.17809713434456

Strategy SelectiveReturns has maximum Sharpe Ratio of 0.5970603777044277 with key (10, 3)
These parameters yield a max drawdown of 17.834203434595377

Strategy SelectiveReturnsLongOnly has maximum Sharpe Ratio of 0.7476470358882656 with key (25, 1)
These parameters yield a max drawdown of 13.670813432287215



### Live Level Testing
1. Testing optimal parameters from backtesting on live levels 
2. Computing optimal parameters from live levels

In [35]:
live_prices = pd.read_pickle(r'other_prices.pkl')
live_prices.head()

,AAPL,AMZN,GOOGL,MSFT
2021-08-19,144.1302,159.0045,135.3431,289.3040
2021-08-20,145.5941,159.6131,137.0883,296.7030
2021-08-23,147.0875,162.9012,139.6938,296.9857
2021-08-24,146.9991,164.8919,140.9107,295.0068
2021-08-25,145.7612,164.5627,141.7262,294.4121


In [36]:
# Test backtesting optimal parameters on live prices 
for strategy in strategies: 
    levels = strategy.compute_strategy_with_max_parameters(live_prices)
    sr = sharpe_ratio(levels)
    mdd = max_drawdown(levels)
    
    print(f'Strategy {strategy.uid} has SR {sr} and MDD {mdd} with backtest optimal parameters of {strategy.max_key}')
    print('')

Strategy ReturnsOnly has SR -0.005243821351830233 and MDD 13.35785093068617 with backtest optimal parameters of ()

Strategy AverageReturn has SR -0.6082886965924733 and MDD 18.206040773775484 with backtest optimal parameters of (25,)

Strategy CrossingAverages has SR -0.28567972887215015 and MDD 16.729568389474082 with backtest optimal parameters of (25, 100)

Strategy SelectiveReturns has SR -0.5594680128839289 and MDD 17.76024720735097 with backtest optimal parameters of (10, 3)

Strategy SelectiveReturnsLongOnly has SR 0.2298456670607178 and MDD 7.406635816441434 with backtest optimal parameters of (25, 1)



In [37]:
for strategy in strategies: 
    strategy.test_parameters(prices = live_prices)
    
    print(f'Strategy {strategy.uid} has maximum Sharpe Ratio of {strategy.max_sharpe_ratio} with key {strategy.max_key}')
    print(f'These parameters yield a max drawdown of {strategy.all_records[strategy.max_key][1]}')
    print('')

Strategy ReturnsOnly has maximum Sharpe Ratio of -0.005243821351830233 with key ()
These parameters yield a max drawdown of 13.35785093068617

Strategy AverageReturn has maximum Sharpe Ratio of 0.20620678980474233 with key (252,)
These parameters yield a max drawdown of 16.41179559180987

Strategy CrossingAverages has maximum Sharpe Ratio of 0.05670574642995853 with key (50, 500)
These parameters yield a max drawdown of 16.46060082112769

Strategy SelectiveReturns has maximum Sharpe Ratio of 0.4116801670837305 with key (252, 3)
These parameters yield a max drawdown of 11.674984758878207

Strategy SelectiveReturnsLongOnly has maximum Sharpe Ratio of 0.46014136030525943 with key (252, 3)
These parameters yield a max drawdown of 9.310348315170216

